pip install "sglang[all]"; pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/

pip install -U torch transformers accelerate trl peft

In [2]:
import dspy
import random
from dspy.datasets import DataLoader
from datasets import load_dataset

# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)
print(CLASSES)
# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
]

random.Random(0).shuffle(raw_data)

['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'getting_virtual_card'

In [3]:
len(CLASSES), CLASSES[:10]

(77,
 ['activate_my_card',
  'age_limit',
  'apple_pay_or_google_pay',
  'atm_support',
  'automatic_top_up',
  'balance_not_updated_after_bank_transfer',
  'balance_not_updated_after_cheque_or_cash_deposit',
  'beneficiary_not_allowed',
  'cancel_transfer',
  'card_about_to_expire'])

In [4]:
unlabeled_trainset = [dspy.Example(text=x.text).with_inputs("text") for x in raw_data[:500]]

unlabeled_trainset[0]

Example({'text': 'What if there is an error on the exchange rate?'}) (input_keys={'text'})

In [5]:
from typing import Literal

classify = dspy.ChainOfThought(f"text -> label:Literal{CLASSES}")
# f"text -> label:Literal{CLASSES}"

In [6]:
from dspy.clients.lm_local import LocalProvider

from dotenv import load_dotenv
load_dotenv()

student_lm_name = "meta-llama/Llama-3.2-1B-Instruct"
student_lm = dspy.LM(model=f"openai/local:{student_lm_name}", provider=LocalProvider(), max_tokens=2000)
teacher_lm = dspy.LM('openai/gpt-4o-mini', max_tokens=3000)

In [17]:
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

teacher_classify = classify.deepcopy()
teacher_classify.set_lm(teacher_lm)

In [18]:
import litellm
litellm.cache = None

optimizer = dspy.BootstrapFinetune(num_threads=16)  # if you *do* have labels, pass metric=your_metric here!
dspy.settings.experimental = True
classify_ft = optimizer.compile(student_classify, teacher=teacher_classify, trainset=unlabeled_trainset)
classify_ft

2025/03/14 03:57:44 INFO dspy.teleprompt.bootstrap_finetune: Preparing the student and teacher programs...
2025/03/14 03:57:44 INFO dspy.teleprompt.bootstrap_finetune: Bootstrapping data...


Average Metric: 500.00 / 500 (100.0%): 100%|██████████| 500/500 [00:00<00:00, 2325.96it/s]

2025/03/14 03:57:44 INFO dspy.evaluate.evaluate: Average Metric: 500 / 500 (100.0%)


2025/03/14 03:57:44 INFO dspy.teleprompt.bootstrap_finetune: Preparing the train data...
2025/03/14 03:57:45 INFO dspy.teleprompt.bootstrap_finetune: Using 500 data points for fine-tuning the model: openai/local:meta-llama/Llama-3.2-1B-Instruct
2025/03/14 03:57:45 INFO dspy.teleprompt.bootstrap_finetune: Starting LM fine-tuning...
2025/03/14 03:57:45 INFO dspy.teleprompt.bootstrap_finetune: 1 fine-tuning job(s) to start
2025/03/14 03:57:45 INFO dspy.teleprompt.bootstrap_finetune: Starting 1 fine-tuning job(s)...
2025/03/14 03:57:45 INFO dspy.teleprompt.bootstrap_finetune: Calling lm.kill() on the LM to be fine-tuned to free up resources. This won't have any effect if the LM is not running.
2025/03/14 03:57:45 INFO dspy.clients.lm_local: No running server to kill.
2025/03/14 03:57:45 INFO dspy.clients.lm_local: Starting local training, will save to /root/.dspy_cache/finetune/c88c0a819a7145bc_vxmr9m_meta-llama-Llama-3.2-1B-Instruct_2025-03-14_03-57-45
2025/03/14 03:57:45 ERROR dspy.clien

predict = Predict(StringSignature(text -> reasoning, label
    instructions='Given the fields `text`, produce the fields `label`.'
    text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text:', 'desc': '${text}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    label = Field(annotation=Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawa

In [19]:
classify_ft.get_lm().launch()

AttributeError: 'RuntimeError' object has no attribute 'launch'